In [10]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd


In [11]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [ ]:
!nvidia-smi


In [ ]:
# TO REMOVE CORRUPTED IMAGES

import os
from PIL import Image

RAW_DATASET_DIR = 'raw_dataset'  # Your original dataset folder

def remove_corrupted_images(dataset_dir):
    removed_files = []
    for class_name in os.listdir(dataset_dir):
        class_path = os.path.join(dataset_dir, class_name)
        if not os.path.isdir(class_path):
            continue
        for img_file in os.listdir(class_path):
            img_path = os.path.join(class_path, img_file)
            try:
                with Image.open(img_path) as img:
                    img.verify()  # Verify if image is corrupted
            except (IOError, SyntaxError) as e:
                print(f"Removing corrupted image: {img_path}")
                os.remove(img_path)
                removed_files.append(img_path)
    return removed_files

removed = remove_corrupted_images(RAW_DATASET_DIR)
print(f"Total corrupted images removed: {len(removed)}")


In [ ]:
# TO CLEAN LABELS
import re

def check_class_names(dataset_dir):
    invalid_names = []
    for class_name in os.listdir(dataset_dir):
        class_path = os.path.join(dataset_dir, class_name)
        if not os.path.isdir(class_path):
            continue
        # Allow only alphanumeric, underscore, and spaces
        if not re.match(r'^[\w\s]+$', class_name):
            invalid_names.append(class_name)
    return invalid_names

invalid_classes = check_class_names(RAW_DATASET_DIR)
if invalid_classes:
    print("Invalid class folder names detected:")
    for name in invalid_classes:
        print(f" - {name}")
else:
    print("All class folder names are clean!")


In [ ]:
# Resizing Images and Converting to JPEG

from PIL import Image
import os

def resize_and_convert(dataset_dir, output_dir, target_size):
    total_images = 0
    total_skipped = 0
    for class_name in os.listdir(dataset_dir):
        class_path = os.path.join(dataset_dir, class_name)
        if not os.path.isdir(class_path):
            continue
        processed_class_path = os.path.join(output_dir, class_name.replace(' ', '_'))
        os.makedirs(processed_class_path, exist_ok=True)

        count = 0
        for img_name in os.listdir(class_path):
            src_path = os.path.join(class_path, img_name)
            try:
                with Image.open(src_path) as img:
                    img = img.convert('RGB')
                    img = img.resize(target_size, Image.Resampling.LANCZOS)
                    base_name = os.path.splitext(img_name)[0]
                    save_path = os.path.join(processed_class_path, base_name + '.jpg')
                    img.save(save_path, 'JPEG', quality=95)
                count += 1
                total_images += 1
            except Exception as e:
                print(f"Skipping invalid image {src_path}: {e}")
                total_skipped += 1
        print(f"Processed {count} images in class '{class_name}'")

    print(f"Total images processed: {total_images}")
    print(f"Total images skipped: {total_skipped}")

# Example usage:
resize_and_convert('raw_dataset', 'processed_dataset', (224, 224))


In [ ]:
import os

def count_images(dataset_dir):
    total_images = 0
    for class_name in os.listdir(dataset_dir):
        class_path = os.path.join(dataset_dir, class_name)
        if not os.path.isdir(class_path):
            continue
        num_images = len([f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        print(f"Class '{class_name}': {num_images} images")
        total_images += num_images
    print(f"Total images in dataset: {total_images}")
    return total_images

count_images('processed_dataset')


In [ ]:
import os

def count_images(dataset_dir):
    total_images = 0
    for class_name in os.listdir(dataset_dir):
        class_path = os.path.join(dataset_dir, class_name)
        if not os.path.isdir(class_path):
            continue
        num_images = len([f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))])
        print(f"Class '{class_name}': {num_images} images")
        total_images += num_images
    print(f"Total images in dataset: {total_images}")
    return total_images

count_images('raw_dataset')

In [ ]:
import os
from PIL import Image

PROCESSED_DATASET_DIR = 'processed_dataset'  # update path if different

def dhash(image, hash_size=8):
    # Use LANCZOS resampling instead of deprecated ANTIALIAS
    image = image.convert('L').resize((hash_size + 1, hash_size), Image.Resampling.LANCZOS)
    pixels = list(image.getdata())
    difference = []
    for row in range(hash_size):
        for col in range(hash_size):
            left_pixel = pixels[row * (hash_size + 1) + col]
            right_pixel = pixels[row * (hash_size + 1) + col + 1]
            difference.append(left_pixel > right_pixel)
    decimal_value = 0
    hex_string = []
    for index, value in enumerate(difference):
        if value:
            decimal_value += 2 ** (index % 8)
        if (index % 8) == 7:
            hex_string.append(hex(decimal_value)[2:].rjust(2, '0'))
            decimal_value = 0
    return ''.join(hex_string)

def remove_duplicates(dataset_dir):
    hashes = set()
    removed_count = 0
    for class_name in os.listdir(dataset_dir):
        class_path = os.path.join(dataset_dir, class_name)
        if not os.path.isdir(class_path):
            continue
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            try:
                with Image.open(img_path) as img:
                    h = dhash(img)
                if h in hashes:
                    print(f"Duplicate found, removing: {img_path}")
                    os.remove(img_path)
                    removed_count += 1
                else:
                    hashes.add(h)
            except Exception as e:
                print(f"Error processing image {img_path}: {e}")
    return removed_count

duplicates_removed = remove_duplicates(PROCESSED_DATASET_DIR)
print(f"Duplicates removed: {duplicates_removed}")


In [ ]:
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img

# Set the processed dataset directory
DATASET_DIR = 'processed_dataset'

# ImageDataGenerator with augmentation settings
augmentor = ImageDataGenerator(
    rotation_range=25,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest"
)

# Initialize total counters
total_original_images = 0
total_augmented_images = 0

# Process each class
for class_name in os.listdir(DATASET_DIR):
    class_path = os.path.join(DATASET_DIR, class_name)
    if not os.path.isdir(class_path):
        continue

    print(f"\n📁 Processing class: {class_name}")
    image_count = 0
    augmented_count = 0

    for img_name in os.listdir(class_path):
        if img_name.startswith('aug_'):
            continue  # Skip already augmented images

        try:
            img_path = os.path.join(class_path, img_name)
            img = load_img(img_path)
            x = img_to_array(img)
            x = x.reshape((1,) + x.shape)

            # Generate 3 augmented images
            i = 0
            for batch in augmentor.flow(
                x,
                batch_size=1,
                save_to_dir=class_path,
                save_prefix='aug',
                save_format='jpeg'
            ):
                i += 1
                augmented_count += 1
                total_augmented_images += 1
                if i >= 3:
                    break

            image_count += 1
            total_original_images += 1
            print(f"✅ Augmented 3 images for: {img_name}")

        except Exception as e:
            print(f"❌ Error augmenting image {img_name}: {e}")

    print(f"📊 Class summary — Original: {image_count}, Augmented: {augmented_count}")

print("\n✅ Augmentation complete")
print(f"🔢 Total original images processed: {total_original_images}")
print(f"📈 Total augmented images created: {total_augmented_images}")


In [ ]:
import os
from PIL import Image
import random
from torchvision import transforms

# Define your augmentation pipeline
augmentation = transforms.Compose([
    transforms.RandomRotation(degrees=20),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1)
])

def find_minority_classes(dataset_dir, threshold=3500):
    minority_classes = []
    print("🔍 Checking classes for minority status based on threshold ≤", threshold)
    for class_name in os.listdir(dataset_dir):
        class_path = os.path.join(dataset_dir, class_name)
        if not os.path.isdir(class_path):
            continue
        num_images = len([name for name in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, name))])
        print(f"Class '{class_name}' has {num_images} images")
        if num_images <= threshold:
            minority_classes.append(class_name)
            print(f"✅ Class '{class_name}' marked as minority class")
    return minority_classes

def augment_minority_classes(dataset_dir, minority_classes, augmentations_per_image=3):
    total_original = 0
    total_augmented = 0

    for class_name in os.listdir(dataset_dir):
        class_path = os.path.join(dataset_dir, class_name)
        if not os.path.isdir(class_path):
            continue

        if class_name not in minority_classes:
            print(f"⏭ Skipping majority class: {class_name}")
            continue
        
        print(f"\n🚀 Augmenting minority class: {class_name}")

        for img_name in os.listdir(class_path):
            if img_name.startswith('aug_') or '_aug' in img_name:
                # Skip already augmented images to avoid duplication
                continue
            
            img_path = os.path.join(class_path, img_name)
            try:
                with Image.open(img_path) as img:
                    total_original += 1
                    base_name, ext = os.path.splitext(img_name)
                    
                    for i in range(augmentations_per_image):
                        augmented_img = augmentation(img)
                        augmented_name = f"{base_name}_aug{i+1}.jpg"
                        augmented_path = os.path.join(class_path, augmented_name)
                        augmented_img.save(augmented_path, 'JPEG', quality=95)
                        total_augmented += 1

                    print(f"✅ Augmented {augmentations_per_image} images for: {img_name}")

            except Exception as e:
                print(f"❌ Error augmenting {img_path}: {e}")

    print("\n✅ Augmentation complete")
    print(f"🔢 Total original images processed for minority classes: {total_original}")
    print(f"📈 Total augmented images created for minority classes: {total_augmented}")

# Set your processed dataset directory path here
PROCESSED_DATASET_DIR = 'processed_dataset'

# Step 1: Find minority classes automatically
minority_classes = find_minority_classes(PROCESSED_DATASET_DIR, threshold=3500)
print("\n🔎 Minority classes detected:", minority_classes)

# Step 2: Augment only minority classes
augment_minority_classes(PROCESSED_DATASET_DIR, minority_classes)


In [ ]:
import os
from PIL import Image
import random
from torchvision import transforms

# Define your augmentation pipeline
augmentation = transforms.Compose([
    transforms.RandomRotation(degrees=20),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.1)
])

def find_minority_classes(dataset_dir, threshold=2000):
    minority_classes = []
    print("🔍 Checking classes for minority status based on threshold ≤", threshold)
    for class_name in os.listdir(dataset_dir):
        class_path = os.path.join(dataset_dir, class_name)
        if not os.path.isdir(class_path):
            continue
        num_images = len([name for name in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, name))])
        print(f"Class '{class_name}' has {num_images} images")
        if num_images <= threshold:
            minority_classes.append(class_name)
            print(f"✅ Class '{class_name}' marked as minority class")
    return minority_classes

def augment_minority_classes(dataset_dir, minority_classes, augmentations_per_image=3):
    total_original = 0
    total_augmented = 0

    for class_name in os.listdir(dataset_dir):
        class_path = os.path.join(dataset_dir, class_name)
        if not os.path.isdir(class_path):
            continue

        if class_name not in minority_classes:
            print(f"⏭ Skipping majority class: {class_name}")
            continue
        
        print(f"\n🚀 Augmenting minority class: {class_name}")

        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            try:
                with Image.open(img_path) as img:
                    total_original += 1
                    base_name, ext = os.path.splitext(img_name)
                    
                    for i in range(augmentations_per_image):
                        augmented_img = augmentation(img)
                        augmented_name = f"{base_name}_aug{i+1}.jpg"
                        augmented_path = os.path.join(class_path, augmented_name)
                        augmented_img.save(augmented_path, 'JPEG', quality=95)
                        total_augmented += 1

                    print(f"✅ Augmented {augmentations_per_image} images for: {img_name}")

            except Exception as e:
                print(f"❌ Error augmenting {img_path}: {e}")

    print("\n✅ Augmentation complete")
    print(f"🔢 Total images processed for minority classes: {total_original}")
    print(f"📈 Total augmented images created for minority classes: {total_augmented}")

# Set your processed dataset directory path here
PROCESSED_DATASET_DIR = 'processed_dataset'

# Step 1: Find minority classes automatically
minority_classes = find_minority_classes(PROCESSED_DATASET_DIR, threshold=2000)
print("\n🔎 Minority classes detected:", minority_classes)

# Step 2: Augment only minority classes
augment_minority_classes(PROCESSED_DATASET_DIR, minority_classes)


In [ ]:
import os
import json

# ======================
# 🔧 CONFIGURATION
# ======================
DATASET_DIR = 'processed_dataset'  # Folder where class subfolders are
LABEL_MAPPING_FILE = 'label_mapping.json'

def encode_labels(dataset_dir, save_mapping_file):
    print("🔍 Scanning dataset for class labels...")

    # Get list of folder names (each folder = one class label)
    class_labels = [d for d in os.listdir(dataset_dir) if os.path.isdir(os.path.join(dataset_dir, d))]
    print(f"🗂 Found {len(class_labels)} classes:")
    for label in class_labels:
        print(f"  - {label}")

    # Sort labels and assign integers
    label_to_int = {label: idx for idx, label in enumerate(sorted(class_labels))}
    print("\n🔢 Label to integer mapping:")
    for label, idx in label_to_int.items():
        print(f"  '{label}' -> {idx}")

    # Save mapping to JSON for future reference
    with open(save_mapping_file, 'w') as f:
        json.dump(label_to_int, f, indent=4)
    print(f"\n💾 Saved label mapping to '{save_mapping_file}'")

    print("\n✅ Label encoding done!")
    return label_to_int

# Run encoding
label_mapping = encode_labels(DATASET_DIR, LABEL_MAPPING_FILE)


In [ ]:
import os
import shutil
import random

# ======================
# 🔧 CONFIGURATION
# ======================
DATASET_DIR = 'processed_dataset'  # Original dataset folder with class subfolders
OUTPUT_DIR = 'dataset_split'       # Folder to create train/, val/, and test/ folders inside
TRAIN_RATIO = 0.7
VAL_RATIO = 0.1
TEST_RATIO = 0.2
SEED = 42  # For reproducibility

random.seed(SEED)

def split_dataset(dataset_dir, output_dir, train_ratio=0.7, val_ratio=0.1, test_ratio=0.2):
    assert abs(train_ratio + val_ratio + test_ratio - 1.0) < 1e-6, "Ratios must sum to 1"

    train_dir = os.path.join(output_dir, 'train')
    val_dir = os.path.join(output_dir, 'val')
    test_dir = os.path.join(output_dir, 'test')

    os.makedirs(train_dir, exist_ok=True)
    os.makedirs(val_dir, exist_ok=True)
    os.makedirs(test_dir, exist_ok=True)

    for class_name in os.listdir(dataset_dir):
        class_path = os.path.join(dataset_dir, class_name)
        if not os.path.isdir(class_path):
            continue

        train_class_dir = os.path.join(train_dir, class_name)
        val_class_dir = os.path.join(val_dir, class_name)
        test_class_dir = os.path.join(test_dir, class_name)

        os.makedirs(train_class_dir, exist_ok=True)
        os.makedirs(val_class_dir, exist_ok=True)
        os.makedirs(test_class_dir, exist_ok=True)

        images = [f for f in os.listdir(class_path) if os.path.isfile(os.path.join(class_path, f))]
        random.shuffle(images)

        n_total = len(images)
        n_train = int(n_total * train_ratio)
        n_val = int(n_total * val_ratio)

        train_images = images[:n_train]
        val_images = images[n_train:n_train + n_val]
        test_images = images[n_train + n_val:]

        for img_name in train_images:
            shutil.copy2(os.path.join(class_path, img_name), os.path.join(train_class_dir, img_name))

        for img_name in val_images:
            shutil.copy2(os.path.join(class_path, img_name), os.path.join(val_class_dir, img_name))

        for img_name in test_images:
            shutil.copy2(os.path.join(class_path, img_name), os.path.join(test_class_dir, img_name))

    print(f"✅ Dataset split completed with {train_ratio*100}% train, {val_ratio*100}% val, and {test_ratio*100}% test.")

# Run the split
split_dataset(DATASET_DIR, OUTPUT_DIR, TRAIN_RATIO, VAL_RATIO, TEST_RATIO)


In [3]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

transform = transforms.Compose([
    transforms.Resize((224, 224)),  # if needed again
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # RGB
])

train_dataset = datasets.ImageFolder('dataset_split/train', transform=transform)
val_dataset = datasets.ImageFolder('dataset_split/val', transform=transform)
test_dataset = datasets.ImageFolder('dataset_split/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)
test_loader = DataLoader(test_dataset, batch_size=32)


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class PlantDiseaseCNN(nn.Module):
    def __init__(self, num_classes):
        super(PlantDiseaseCNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.conv4 = nn.Conv2d(128, 256, kernel_size=3, padding=1)

        self.dropout = nn.Dropout(0.35)

        self.fc1 = nn.Linear(256 * 14 * 14, 512)  # After 2 poolings, 224→112→56→28→14
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # [B, 32, 112, 112]
        x = self.pool(F.relu(self.conv2(x)))  # [B, 64, 56, 56]
        x = self.pool(F.relu(self.conv3(x)))  # [B, 128, 28, 28]
        x = self.pool(F.relu(self.conv4(x)))  # [B, 256, 14, 14]

        x = x.view(-1, 256 * 14 * 14)         # Flatten
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x


In [5]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Using device: cuda


In [1]:
import torch
print(torch.cuda.is_available())           # Should be True

True


In [2]:
import torch

print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")



PyTorch version: 2.7.1+cu118
CUDA available: True
GPU device name: NVIDIA RTX A2000 Laptop GPU


In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
import time

# Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model
model = PlantDiseaseCNN(num_classes=15).to(device)

# Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10

# Training Loop
for epoch in range(num_epochs):
    start_time = time.time()

    # ======== TRAINING =========
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    train_loader_tqdm = tqdm(train_loader, desc=f"🚂 Epoch {epoch+1}/{num_epochs} - Training", leave=False)
    for images, labels in train_loader_tqdm:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)  # 🔧 accumulate weighted loss
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_accuracy = 100 * correct / total
    train_loss = running_loss / total  # 🔧 average loss

    # ======== VALIDATION =========
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        val_loader_tqdm = tqdm(val_loader, desc=f"🔍 Epoch {epoch+1}/{num_epochs} - Validating", leave=False)
        for images, labels in val_loader_tqdm:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item() * images.size(0)  # 🔧 accumulate weighted loss
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()

    val_accuracy = 100 * val_correct / val_total
    val_loss = val_loss / val_total  # 🔧 average loss
    duration = time.time() - start_time

    print(f"✅ Epoch {epoch+1}/{num_epochs} | "
          f"Train Loss: {train_loss:.4f} | Train Acc: {train_accuracy:.2f}% | "
          f"Val Loss: {val_loss:.4f} | Val Acc: {val_accuracy:.2f}% | "
          f"🕒 Time: {duration:.2f}s")

# Save Model
torch.save(model.state_dict(), 'plant_disease_cnn.pth')
print("💾 Model training complete and saved as 'plant_disease_cnn.pth'")


✅ Epoch 1/10 | Train Loss: 0.7675 | Train Acc: 73.96% | Val Loss: 0.3042 | Val Acc: 89.61% | 🕒 Time: 288.37s


✅ Epoch 2/10 | Train Loss: 0.2863 | Train Acc: 90.45% | Val Loss: 0.2107 | Val Acc: 92.81% | 🕒 Time: 279.00s


✅ Epoch 3/10 | Train Loss: 0.1806 | Train Acc: 93.79% | Val Loss: 0.1918 | Val Acc: 93.60% | 🕒 Time: 279.74s


✅ Epoch 4/10 | Train Loss: 0.1313 | Train Acc: 95.73% | Val Loss: 0.1934 | Val Acc: 93.88% | 🕒 Time: 284.37s


✅ Epoch 5/10 | Train Loss: 0.1110 | Train Acc: 96.26% | Val Loss: 0.1739 | Val Acc: 94.61% | 🕒 Time: 291.30s


✅ Epoch 6/10 | Train Loss: 0.0883 | Train Acc: 97.11% | Val Loss: 0.1861 | Val Acc: 94.74% | 🕒 Time: 282.59s


✅ Epoch 7/10 | Train Loss: 0.0798 | Train Acc: 97.50% | Val Loss: 0.1579 | Val Acc: 95.60% | 🕒 Time: 275.80s


✅ Epoch 8/10 | Train Loss: 0.0692 | Train Acc: 97.78% | Val Loss: 0.2127 | Val Acc: 94.30% | 🕒 Time: 264.51s


✅ Epoch 9/10 | Train Loss: 0.0690 | Train Acc: 97.91% | Val Loss: 0.1644 | Val Acc: 95.73% | 🕒 Time: 264.72s


✅ Epoch 10/10 | Train Loss: 0.0656 | Train Acc: 98.07% | Val Loss: 0.1797 | Val Acc: 95.04% | 🕒 Time: 307.18s
💾 Model training complete and saved as 'plant_disease_cnn.pth'


In [9]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import numpy as np
import torch
from torchvision import transforms
from tqdm import tqdm
from collections import Counter
import pandas as pd
import torch.nn.functional as F

# Ensure results directory exists
os.makedirs("results", exist_ok=True)

# ✅ Reload trained model
model = PlantDiseaseCNN(num_classes=15).to(device)
model.load_state_dict(torch.load('plant_disease_cnn.pth'))
model.eval()

# ✅ Predict on Test Data
all_preds = []
all_labels = []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="🔍 Testing"):
        images = images.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.numpy())

# ✅ 1. Classification Report
print("\n📊 Classification Report:\n")
report = classification_report(all_labels, all_preds, target_names=test_dataset.classes)
print(report)

# Save report as text file
with open("results/classification_report.txt", "w") as f:
    f.write(report)

# ✅ 2. Overall Test Accuracy
acc = accuracy_score(all_labels, all_preds)
print(f"\n✅ Test Accuracy: {acc * 100:.2f}%")

# ✅ 3. Confusion Matrix Heatmap
cm = confusion_matrix(all_labels, all_preds)
plt.figure(figsize=(14, 10))
sns.heatmap(cm, annot=True, fmt="d", cmap="Blues",
            xticklabels=test_dataset.classes,
            yticklabels=test_dataset.classes,
            cbar_kws={'label': 'Count'},
            annot_kws={"size": 7})
plt.title("📊 Confusion Matrix")
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
plt.xticks(rotation=45, ha='right')
plt.yticks(rotation=0)
plt.tight_layout()
plt.savefig("results/confusion_matrix.png")
plt.close()

# ✅ 4. Per-Class Accuracy Bar Plot
class_accuracy = cm.diagonal() / cm.sum(axis=1)
plt.figure(figsize=(14, 5))
sns.barplot(x=test_dataset.classes, y=class_accuracy)
plt.xticks(rotation=90)
plt.ylabel("Accuracy")
plt.title("📈 Per-Class Accuracy")
plt.ylim(0, 1.05)
plt.tight_layout()
plt.savefig("results/per_class_accuracy.png")
plt.close()

# ✅ 5. Misclassified Images Viewer (and Save)
def show_misclassified_images(model, loader, class_names, device, num_images=10):
    model.eval()
    misclassified = []
    
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)

            for img, pred, label in zip(images, preds, labels):
                if pred != label:
                    misclassified.append((img.cpu(), pred.cpu(), label.cpu()))
                if len(misclassified) >= num_images:
                    break
            if len(misclassified) >= num_images:
                break

    plt.figure(figsize=(15, 6))
    for i, (img, pred, label) in enumerate(misclassified):
        plt.subplot(2, 5, i + 1)
        plt.imshow(img.permute(1, 2, 0).numpy() * 0.5 + 0.5)
        plt.title(f"Pred: {class_names[pred]}\nTrue: {class_names[label]}", fontsize=8)
        plt.axis('off')
    plt.suptitle("❌ Misclassified Examples", fontsize=16)
    plt.tight_layout()
    plt.savefig("results/misclassified_examples.png")
    plt.close()

show_misclassified_images(model, test_loader, test_dataset.classes, device)

# ✅ 6. Test Class Distribution
test_labels = [label for _, label in test_dataset]
label_counts = Counter(test_labels)

plt.figure(figsize=(14, 5))
sns.barplot(x=[test_dataset.classes[i] for i in label_counts.keys()],
            y=list(label_counts.values()))
plt.xticks(rotation=90)
plt.title("📦 Class Distribution in Test Dataset")
plt.ylabel("Number of Samples")
plt.tight_layout()
plt.savefig("results/class_distribution.png")
plt.close()

# ✅ 7. Save Predictions to CSV
predictions = []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        outputs = model(images)
        probabilities = F.softmax(outputs, dim=1)
        _, predicted = torch.max(outputs, 1)

        for i in range(len(images)):
            image_id = test_dataset.imgs[i][0].split('/')[-1]
            predicted_class = test_dataset.classes[predicted[i]]
            predicted_probability = probabilities[i][predicted[i]].item()
            predictions.append([image_id, predicted_class, predicted_probability])

df_predictions = pd.DataFrame(predictions, columns=['image_id', 'predicted_class', 'predicted_probability'])
df_predictions.to_csv('results/test_predictions.csv', index=False)
print("✅ Predictions saved to 'results/test_predictions.csv'.")


🔍 Testing: 100%|██████████| 526/526 [00:42<00:00, 12.39it/s]



📊 Classification Report:

                                             precision    recall  f1-score   support

              Pepper__bell___Bacterial_spot       0.97      0.93      0.95      1313
                     Pepper__bell___healthy       0.98      0.96      0.97      1902
                      Potato___Early_blight       0.98      0.98      0.98       816
                       Potato___Late_blight       0.92      0.97      0.94       720
                           Potato___healthy       0.96      0.96      0.96       752
                      Tomato_Bacterial_spot       0.97      0.95      0.96      1367
                        Tomato_Early_blight       0.89      0.87      0.88       720
                         Tomato_Late_blight       0.96      0.92      0.94      1243
                           Tomato_Leaf_Mold       0.91      0.98      0.94      1257
                  Tomato_Septoria_leaf_spot       0.93      0.95      0.94      1185
Tomato_Spider_mites_Two_spotted_spide

C:\Users\Muhammad Iqbal\AppData\Local\Temp\ipykernel_10168\283138403.py:59: UserWarning: Glyph 128202 (\N{BAR CHART}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\Muhammad Iqbal\AppData\Local\Temp\ipykernel_10168\283138403.py:60: UserWarning: Glyph 128202 (\N{BAR CHART}) missing from font(s) DejaVu Sans.
  plt.savefig("results/confusion_matrix.png")
C:\Users\Muhammad Iqbal\AppData\Local\Temp\ipykernel_10168\283138403.py:71: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\Muhammad Iqbal\AppData\Local\Temp\ipykernel_10168\283138403.py:72: UserWarning: Glyph 128200 (\N{CHART WITH UPWARDS TREND}) missing from font(s) DejaVu Sans.
  plt.savefig("results/per_class_accuracy.png")
C:\Users\Muhammad Iqbal\AppData\Local\Temp\ipykernel_10168\283138403.py:101: UserWarning: Glyph 10060 (\N{CROSS MARK}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
C:\Users\Muhammad Iqbal\AppData\Local\Temp\ipykernel

✅ Predictions saved to 'results/test_predictions.csv'.
